In [ ]:
import numpy as np
import cv2
source=cv2.VideoCapture('Samples/road_drive.avi')
kernel=np.array([[-1,0,1],
                 [-2,0,2],
                 [-1,0,1]],np.float64)
while(True):
    
    ret,img=source.read()
    h,w =img.shape[:2] 
    
    frameCx=w//2
    cv2.line(img,(frameCx,0),(frameCx,h),(0,255,0),2)
    
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    conv=cv2.filter2D(gray,-1,kernel)
    ret, thresh = cv2.threshold(conv, 100, 255, 0)

    thresh[0:130,:]=0
    kernels = np.ones((2,2),np.uint8)
    
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernels)
    contours, hierarchy = cv2.findContours(opening, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    for cnt in contours:
        area=cv2.contourArea(cnt)
        #print(area)
        if(area>60):
            cv2.drawContours(img, [cnt], -1, (0,255,255), 3)
            #x,y,w,h = cv2.boundingRect(cnt)
            #img = cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)            
            rect = cv2.minAreaRect(cnt)
            box = cv2.boxPoints(rect)
            box = np.intp(box)
            cv2.drawContours(img,[box],0,(0,0,255),2)
            M = cv2.moments(cnt)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            distance=frameCx-cx
            cv2.line(img,(frameCx,cy),(cx,cy),(0,255,0),2)
            cv2.putText(img, 'Distance:'+str(distance), (cx,cy), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,255),2 )
            
            
            
            
    cv2.imshow('LIVE',img)
    cv2.imshow('conv',conv)
   # cv2.imshow('thresh',thresh)
    cv2.imshow('open',opening)
    key=cv2.waitKey(100)

    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()